<a href="https://colab.research.google.com/github/Manisha0597/Customer-Sentiment-Alert/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚨 Customer Sentiment Alert (CSA Agent)


In [ ]:
# Step 1: Install dependencies
!pip install transformers scikit-learn

In [ ]:
# Step 2: Full CSA Agent code
from transformers import pipeline
import math

# Initialize Sentiment Pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

# Functions
def analyze_sentiment(text):
    res = sentiment_pipeline(text)[0]
    return {'label': res['label'], 'score': float(res['score'])}

def urgency_score(sentiment, reach=1, is_mention=False):
    label = sentiment.get('label', 'NEUTRAL').upper()
    score = sentiment.get('score', 0.0)
    base = 0.0
    if label in ('NEGATIVE','LABEL_0','NEG'):
        base = 0.6 + 0.4*score
    elif label in ('POSITIVE','LABEL_1','POS'):
        base = 0.1*score
    else:
        base = 0.2
    reach_factor = math.log1p(reach)/10.0
    mention_factor = 0.15 if is_mention else 0.0
    urgency = min(1.0, base + reach_factor + mention_factor)
    return round(urgency,3)

# Sample Feedbacks
feedbacks = [
    {'text':'I love the product, it\'s amazing!','reach':10,'is_mention':False},
    {'text':'This is terrible, I hate the service.','reach':150,'is_mention':True},
    {'text':'The app is okay, but it could be faster.','reach':50,'is_mention':False},
    {'text':'Support never responded to my issue, very disappointed!','reach':200,'is_mention':True},
]

# Run Demo
print('🚨 Customer Sentiment Alert\n')
for fb in feedbacks:
    sentiment = analyze_sentiment(fb['text'])
    urgency = urgency_score(sentiment, reach=fb['reach'], is_mention=fb['is_mention'])
    print(f"Feedback: {fb['text']}")
    print(f"Sentiment: {sentiment}")
    print(f"Urgency Score: {urgency}\n")


# Visualization


In [ ]:
# Step 3: Visualize Results
import pandas as pd
import matplotlib.pyplot as plt

# Prepare data
data = []
for fb in feedbacks:
    sentiment = analyze_sentiment(fb['text'])
    urgency = urgency_score(sentiment, reach=fb['reach'], is_mention=fb['is_mention'])
    data.append({
        "Feedback": fb['text'],
        "Sentiment": sentiment['label'],
        "Urgency Score": urgency
    })

df = pd.DataFrame(data)

# Display Table
print("### 🚨 Customer Sentiment Alert Results Table")
display(df)

# Bar Chart
plt.figure(figsize=(10,5))
plt.barh(df['Feedback'], df['Urgency Score'], color='tomato')
plt.xlabel('Urgency Score')
plt.title('🚨 Customer Sentiment Alert - Urgency Score for Feedbacks')
plt.xlim(0,1.0)
plt.show()
